<a href="https://colab.research.google.com/github/Tejaswi-kashyap-006/llms/blob/main/Speculative_decoding_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers nvidia-ml-py3 accelerate bitsandbytes

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.8 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=290facbc5e6287ae3a7bf76df4779746e03ceb8c55427c466cfd7131d42c6d00
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


Run this cell every time you restart the notebook.

In [ ]:
from pynvml import *
import time

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

# Llama 2 13B (main) & TinyLlama 1.1B (draft)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
set_seed(42)  # For reproducibility

checkpoint = "meta-llama/Llama-2-13b-hf"
assistant_checkpoint = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=bnb_config)


prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")


duration = 0.0
total_length = 0
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Prompt --- 9.75 tokens/second ---
GPU memory occupied: 9341 MB.
None
Prompt --- 10.339 tokens/second ---
GPU memory occupied: 9341 MB.
None
Prompt --- 10.38 tokens/second ---
GPU memory occupied: 9341 MB.
None
Prompt --- 10.347 tokens/second ---
GPU memory occupied: 9341 MB.
None
Prompt --- 10.525 tokens/second ---
GPU memory occupied: 9341 MB.
None
Average --- 10.261 tokens/second ---


With speculativate decoding:

In [ ]:
duration = 0.0
total_length = 0

assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint, device_map="auto")

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, assistant_model=assistant_model, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Prompt --- 15.79 tokens/second ---
GPU memory occupied: 13507 MB.
None
Prompt --- 12.769 tokens/second ---
GPU memory occupied: 13521 MB.
None
Prompt --- 17.382 tokens/second ---
GPU memory occupied: 13657 MB.
None
Prompt --- 11.326 tokens/second ---
GPU memory occupied: 13657 MB.
None
Prompt --- 13.111 tokens/second ---
GPU memory occupied: 13679 MB.
None
Average --- 14.306 tokens/second ---


# Gemma 7B (main) & Gemma 2B (draft)



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
set_seed(42)  # For reproducibility

checkpoint = "google/gemma-7b"
assistant_checkpoint = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=bnb_config)


prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")


duration = 0.0
total_length = 0
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Prompt --- 14.085 tokens/second ---
GPU memory occupied: 8111 MB.
None
Prompt --- 15.224 tokens/second ---
GPU memory occupied: 8111 MB.
None
Prompt --- 15.268 tokens/second ---
GPU memory occupied: 8111 MB.
None
Prompt --- 15.279 tokens/second ---
GPU memory occupied: 8111 MB.
None
Prompt --- 15.353 tokens/second ---
GPU memory occupied: 8111 MB.
None
Average --- 15.026 tokens/second ---


With speculativate decoding:

In [ ]:
duration = 0.0
total_length = 0

assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint, device_map="cuda")

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, assistant_model=assistant_model, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Prompt --- 9.464 tokens/second ---
GPU memory occupied: 18583 MB.
None
Prompt --- 27.423 tokens/second ---
GPU memory occupied: 18585 MB.
None
Prompt --- 11.27 tokens/second ---
GPU memory occupied: 18587 MB.
None
Prompt --- 8.885 tokens/second ---
GPU memory occupied: 18587 MB.
None
Prompt --- 29.65 tokens/second ---
GPU memory occupied: 18593 MB.
None
Average --- 13.258 tokens/second ---


# Mixtral-8x7B (main) & Mistral 7B (draft)



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
set_seed(42)  # For reproducibility

checkpoint = "mistralai/Mixtral-8x7B-v0.1"
assistant_checkpoint = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=bnb_config)


prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")


duration = 0.0
total_length = 0
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

Prompt --- 6.897 tokens/second ---
GPU memory occupied: 25331 MB.
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 6.933 tokens/second ---
GPU memory occupied: 25331 MB.
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 6.969 tokens/second ---
GPU memory occupied: 25331 MB.
None
Prompt --- 6.988 tokens/second ---
GPU memory occupied: 25331 MB.
None
Average --- 6.923 tokens/second ---


With speculativate decoding:

In [ ]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
set_seed(42)  # For reproducibility

checkpoint = "mistralai/Mixtral-8x7B-v0.1"
assistant_checkpoint = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="cuda", quantization_config=bnb_config)


prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")

duration = 0.0
total_length = 0

assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint, device_map="cuda", quantization_config=bnb_config)

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, assistant_model=assistant_model, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 9.432 tokens/second ---
GPU memory occupied: 30233 MB.
None


...

Prompt --- 4.112 tokens/second ---
GPU memory occupied: 30233 MB.
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 9.166 tokens/second ---
GPU memory occupied: 30233 MB.
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt --- 6.312 tokens/second ---
GPU memory occupied: 30249 MB.
None
Prompt --- 5.606 tokens/second ---
GPU memory occupied: 30249 MB.
None
Average --- 6.288 tokens/second ---


# Pythia 12B (main) & Pythia 70M (draft)



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig
set_seed(42)  # For reproducibility

checkpoint = "EleutherAI/pythia-12b-deduped"
assistant_checkpoint = "EleutherAI/pythia-70m-deduped"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", quantization_config=bnb_config)


prompt = []
prompt.append("Tell me about gravity.")
prompt.append("What is AI?")
prompt.append("Write an essay about intelligence.")
prompt.append("Cite 20 famous people.")
prompt.append("Give me the recipe for the best chicken curry.")


duration = 0.0
total_length = 0
for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 15.115 tokens/second ---
GPU memory occupied: 8945 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 15.431 tokens/second ---
GPU memory occupied: 8965 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 15.561 tokens/second ---
GPU memory occupied: 8985 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 15.51 tokens/second ---
GPU memory occupied: 9005 MB.
None
Prompt --- 15.729 tokens/second ---
GPU memory occupied: 9005 MB.
None
Average --- 15.467 tokens/second ---


With speculativate decoding:

In [ ]:
duration = 0.0
total_length = 0

assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint, device_map="auto")

for i in range(len(prompt)):
  model_inputs = tokenizer(prompt[i], return_tensors="pt").to("cuda:0")
  start_time = time.time()
  output = model.generate(**model_inputs, assistant_model=assistant_model, max_length=500)[0]
  duration += float(time.time() - start_time)
  total_length += len(output)
  tok_sec_prompt = round(len(output)/float(time.time() - start_time),3)
  print("Prompt --- %s tokens/second ---" % (tok_sec_prompt))
  print(print_gpu_utilization())

tok_sec = round(total_length/duration,3)
print("Average --- %s tokens/second ---" % (tok_sec))

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 79.5 tokens/second ---
GPU memory occupied: 9645 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 29.751 tokens/second ---
GPU memory occupied: 9653 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 96.337 tokens/second ---
GPU memory occupied: 9653 MB.
None


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Prompt --- 75.047 tokens/second ---
GPU memory occupied: 9653 MB.
None
Prompt --- 50.522 tokens/second ---
GPU memory occupied: 9653 MB.
None
Average --- 55.748 tokens/second ---
